In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from os import listdir
from os.path import isfile, join

In [2]:
mypath = '../medpoisk/htmls/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
words = ['мм', 'рт', 'ст', 'г', 'м', 'мл', 'кг', 'мг', 'см', 'ч', 'е', 'х', 'a', 'd', 'b'] 
set_stopWords = stopwords.words("russian") + words
set_stopWords = set(set_stopWords)

In [4]:
def clearHtml(fileName):
    text = ''
    with open(mypath + fileName) as file:
        text = file.read()
    soup = BeautifulSoup(text, 'lxml')
    
    allRows = soup.find_all("div", {"class": "row"})
    res = ''
    for row in allRows[2:]:
        desease_content = row
        for i in desease_content.find_all("div", {"class": "ads-top"}):
            i.decompose()

        for i in desease_content.find_all("a"):
            i.decompose()

        for i in desease_content.find_all("h3"):
            i.decompose()

        for i in desease_content.find_all("div", {"id": "accordion"}):
            i.decompose()
        tmp = desease_content.text
        if('Варианты:' in tmp):
            continue
            
        stop = ['др. греч.', 'греч.', 'лат.', 'др.', 'и т. д', 'и т.д', 'т. е.', 'т.е.', 'т.н.', 'т. н.', 'т.п.', 'т. п.',
                'т. д', 'т.д', 'мм. рт. ст.', 'мм рт. ст.', 'мм.рт.ст', 'МЕДПОИСК.РУ', 'Контактная форма ниже']
        for i in stop:
            tmp = tmp.replace(i, '')
        tmp = tmp.replace('?', '.')
        tmp = re.sub('\d+', ' ', tmp)
        tmp = re.sub('[^\w.]', ' ', tmp)
        res += tmp
    
    res = res.strip()
    final_s = ''
    sentences = res.split('.')
    for sen in sentences:
        sen = sen.strip()
        for tok in sen.split(' '):
            tok = tok.strip()
            if(tok.lower() not in set_stopWords):
                final_s += (tok + ' ')
        final_s += '.'
    
    return final_s

In [5]:
allText = ''
for file in onlyfiles:
    sample = clearHtml(file)
    allText += sample
#     allText += '.'

In [6]:
len(allText.split('.'))

10288

In [7]:
len(allText)

1010476

In [8]:
# with open('medpoisk_text.txt', 'w+') as f:
#     f.write(allText)